In [1]:
import numpy as np
import pandas as pd
y=[0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0] # only for one sample, the highest one is for digit 2
t=[0,0,1,0,0,0,0,0,0,0]#supposing 2 is the correct answer

In [3]:
def mean_squared_error(y,t):
    return 0.5*np.sum((y-t)**2)

In [5]:
mean_squared_error(np.array(y),np.array(t))

0.09750000000000003

In [7]:
y2=[0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0] # the highest one is digit 7
mean_squared_error(np.array(y2),np.array(t))

0.5975

In [9]:
def cross_entropy_error(y,t):
    delta=1e-7 # avoid negative inf
    return -np.sum(t*np.log(y+delta))

In [11]:
cross_entropy_error(np.array(y),np.array(t)) # the smaller the index is, the better the result is

0.510825457099338

In [13]:
cross_entropy_error(np.array(y2),np.array(t))

2.302584092994546

In [15]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
mnist=fetch_openml('mnist_784',version=1)
X,y=mnist['data'],mnist['target']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [17]:
print(X_train.shape)

(56000, 784)


In [19]:
print(y_train.shape)

(56000,)


In [21]:
# convert y_train to one hot coded label
int_y_train=y_train.astype(int)
oh_y_train=np.eye(int_y_train)
print(oh_y_train.shape)

TypeError: 'Series' object cannot be interpreted as an integer

In [23]:
print(type(y_test))

<class 'pandas.core.series.Series'>


In [25]:
npar_y_train=int_y_train.values
oh_y_train=np.eye(10)[npar_y_train]
print(oh_y_train.shape)

(56000, 10)


In [27]:
# the second way to convert y_test
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder(sparse_output=False)
y_train_oh=encoder.fit_transform(npar_y_train.reshape(-1,1))
print(y_train_oh.shape)

(56000, 10)


In [29]:
train_size=X_train.shape[0] #row number
batch_size=10
batch_mask=np.random.choice(train_size,batch_size)
X_train_batch=X_train[batch_mask]
y_train_batch=y_train[batch_mask] # mistake！ I should use y_train_oh or oh_y_train here

KeyError: "None of [Index([4816, 24057, 50800, 17930, 10389, 28128, 10370, 8950, 29706, 40788], dtype='int32')] are in the [columns]"

In [31]:
'''
This happens because DataFrames in Pandas use label-based indexing by default, 
whereas NumPy arrays use integer-based indexing.
'''
# solution is to explicitly use .iloc for row-based integer indexing 
train_size=X_train.shape[0] #row number
batch_size=10
batch_mask=np.random.choice(train_size,batch_size)
X_train_batch=X_train.iloc[batch_mask] # the data type of X_train here is DataFrame
y_train_batch=y_train_oh[batch_mask] 

In [33]:
def cross_entropy_error(y,t):# the data type of  y and t here requires to be NumpyArray
    if y.ndim==1: # deal with one piece of data
        t=t.reshape(1,t.size)
        y=y.reshape(1,y.size)
        print(t)
        print(y)
    batch_size=y.shape[0]
    return -np.sum(t*np.log(y+1e-7))/batch_size

In [35]:
y=np.array([0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]) # only for one sample, the highest one is for digit 2
t=np.array([0,0,1,0,0,0,0,0,0,0])
cross_entropy_error(y,t) # why there is no print?

[[0 0 1 0 0 0 0 0 0 0]]
[[0.1  0.05 0.6  0.   0.05 0.1  0.   0.1  0.   0.  ]]


0.510825457099338

In [37]:
print(y.ndim)

1


In [39]:
y_re=y.reshape(1,y.size)
print(y_re)

[[0.1  0.05 0.6  0.   0.05 0.1  0.   0.1  0.   0.  ]]


In [41]:
print(y_re.ndim)

2


In [153]:
def cross_entropy_error(y,t):
    if y.ndim ==1 or t.ndim==1: # consider two situations
        t=t.reshape(1,t.size)
        y=y.reshape(1,y.size)
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size),t]+1e-7))/batch_size
    # it took me a lot of time to understand y[np.arrange(batch_size,t)]
'''
just figure this example out and you will totally understand it
y = [[0.1, 0.7, 0.2],   # Predicted probabilities for sample 1
     [0.3, 0.4, 0.3],   # Predicted probabilities for sample 2
     [0.2, 0.2, 0.6]]   # Predicted probabilities for sample 3
t = [1, 2, 0]           # True labels for the 3 samples
batch_size = 3
np.arange(batch_size):  [0, 1, 2]
y[np.arange(batch_size), t]: [0.7, 0.3, 0.2]
'''

'\njust figure this example out and you will totally understand it\ny = [[0.1, 0.7, 0.2],   # Predicted probabilities for sample 1\n     [0.3, 0.4, 0.3],   # Predicted probabilities for sample 2\n     [0.2, 0.2, 0.6]]   # Predicted probabilities for sample 3\nt = [1, 2, 0]           # True labels for the 3 samples\nbatch_size = 3\nnp.arange(batch_size):  [0, 1, 2]\ny[np.arange(batch_size), t]: [0.7, 0.3, 0.2]\n'

In [45]:
def numerical_differentiation(f,x):
    h=10e-50 # very small digit
    return (f(x+h)-f(x))/h

In [47]:
def numerical_diff(f,x):
    h=1e-4
    return (f(x+h)-f(x-h))/(2*h)# central difference

In [49]:
def function_1(x):
    return 0.01*x**2+0.1*x

In [51]:
numerical_diff(function_1,5)

0.1999999999990898

In [53]:
numerical_diff(function_1,10)

0.2999999999986347

In [55]:
def function_2(x):
    return x[0]**2+x[1]**2
    # return np.sum(x**2)

In [57]:
# partial derivative
def function_tmp1(x0):
    return x0*x0+4.0*2.0

In [59]:
numerical_diff(function_tmp1,3.0)

6.00000000000378

In [61]:
def function_tmp2(x1):
    return 3.0*2.0+x1*x1

In [63]:
numerical_diff(function_tmp2,4.0)

7.999999999999119

In [65]:
# gradient
def numerical_gradient(f,x): # input x is not a digit, but a vector!
    h=1e-4 #0.0001
    grad=np.zeros_like(x) #生成和x形状相同的数组
    for idx in range(x.size):
        tmp_val=x[idx]
        # the calculation of f(x+h)
        x[idx]=tmp_val+h
        fxh1 = f(x)
        # the calulation of f(x-h)
        x[idx]=tmp_val-h
        fxh2 = f(x)
        grad[idx]=(fxh1-fxh2)/(2*h)
        #central difference
    return grad

In [67]:
numerical_gradient(function_2,np.array([3.0,4.0]))

array([6., 8.])

In [69]:
def gradient_descent(f,init_x,lr=0.01,step_num=100): # lr, learning rate is a hyper parameter here
    x=init_x
    for i in range(step_num):
        grad=numerical_gradient(f,x)
        x-=lr*grad # x0=x0-lr*grad
    return x
#the implementation of gradient descent

In [71]:
def function_2(x):
    return x[0]**2+x[1]**2
init_x = np.array([-3.0,4.0])
gradient_descent(function_2,init_x=init_x,lr=0.1,step_num=100)

array([-0.0005, -0.0005])

In [73]:
#try different learning rate (hyper parameter)
# learning rate=10.0
init_x = np.array([-3.0,4.0])
gradient_descent(function_2,init_x=init_x,lr=10,step_num=100)

array([-2.58985795e+13, -1.29524822e+12])

In [75]:
# learning rate=1e-10
init_x = np.array([-3.0,4.0])
gradient_descent(function_2,init_x=init_x,lr=1e-10,step_num=100)

array([-3.00999994,  3.98999992])

In [77]:
def softmax(x):
    x_max=np.max(x)
    x_new=x-x_max #avoid overflow
    exp_x=np.exp(x_new)
    sum_exp_x=np.sum(exp_x)
    y=exp_x/sum_exp_x
    return y

In [163]:
def cross_entropy_error(y,t):
    if y.ndim==1 or t.ndim==1: #1-D numpyArray and two situations
        t.reshape(1,t.size)
        y.reshape(1,y.size)
    batch_size=y.shape[0] # the row number 
    return -np.sum(t*np.log(y+1e-7))/batch_size

In [81]:
y=np.array([[1,2],
           [3,4],
           [5,6]])
print(y.shape)
print(y.shape[0])# the output of shape is a tuple!

(3, 2)
3


In [83]:
W=np.random.randn(2,3)
print(W)

[[ 2.23643446 -0.69837181  2.00259307]
 [-0.21100293  0.61444281 -0.12267541]]


In [85]:
class simpleNet:#define a class类
    def __init__(self):
        self.W=np.random.randn(2,3) # this is a constructor of class simpleNet Gaussian distribution
        # there is no way to adjust the random state, which means we will get a various output each time we run this code
        #just like a start point of gradient descent
    def predict(self,x):
        return np.dot(x,self.W)
    def loss(self,x,t):
        z=self.predict(x)
        y=softmax(z) # the activating function is softmax(exp_a/sum_exp_a)
        loss = cross_entropy_error(y,t) # loss function
        return loss

In [87]:
net = simpleNet() # instantiate
print(net.W) # weight parameters

[[-0.33048457  2.55696978  0.84154757]
 [ 1.24943231  2.16893274  0.7920311 ]]


In [89]:
x=np.array([0.6,0.9])# x here means a vector so there is no need in capitalizing it 
p=net.predict(x)
print(p)

[0.92619834 3.48622133 1.21775654]


In [91]:
np.argmax(p)

1

In [93]:
t=np.array([1,0,0])# correct answer tag
net.loss(x,t)

0.9087305054392963

In [95]:
def f(W):
    return net.loss(x,t) # a method .loss() from class simpleNet is encapsulated as a new function
# f=lambda W : net.loss(x,t)
dW = numerical_gradient(f,net.W)
print(dW)

IndexError: index 2 is out of bounds for axis 0 with size 2

In [97]:
# gradient for net.W mutiple dimension numpy array/ matrix
def numerical_gradient_W(f,W): # we need to modify the original numerical gradient function to cater to the input of W matrix
    h=1e-4 #0.0001
    grad=np.zeros_like(W) #生成和W形状相同的matrix! to store the calculation output
    for idx1 in range(W.shape[0]): 
        for idx2 in range(W.shape[1]):
            tmp_val=W[idx1][idx2] # first to locate the entry we want to calcualte
            # the calculation of f(x+h)
            W[idx1][idx2]=tmp_val+h
            fxh1 = f(x)
            # the calulation of f(x-h)
            W[idx1][idx2]=tmp_val-h
            fxh2 = f(x)
            grad[idx1][idx2]=(fxh1-fxh2)/(2*h)
            #central difference
    return grad

In [99]:
y=np.array([[1,2],
           [3,4],
           [5,6]])
print(y[0][0]) # get entry from a numpy array matrix

1


In [101]:
dW = numerical_gradient_W(f,net.W)
print(dW)

[[-0.1869061   0.16938085  0.01752688]
 [-0.28035916  0.25407177  0.02629104]]


In [165]:
class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size,weight_init_std=0.01):
        self.params={} # an empty dictionary
        self.params['W1']=weight_init_std*np.random.randn(input_size,hidden_size) # backslash merely for line continuation
        self.params['b1']=np.zeros(hidden_size)
        self.params['W2']=weight_init_std*np.random.randn(hidden_size,output_size) # np.random.randn will create a new matrix
        self.params['b2']=np.zeros(output_size)
    def predict(self,x):
        W1,W2 = self.params['W1'],self.params['W2']
        b1,b2 = self.params['b1'],self.params['b2']
        a1=np.dot(x,W1)+b1
        Z1=sigmoid(a1)
        a2=np.dot(Z1,W2)+b2
        y=softmax(a2)
        #no return here!!!!
        return y
    def loss(self,x,t):
        y=self.predict(x)
        return cross_entropy_error(y,t)
    def accuracy(self,x,t):
        y=self.predict(x)
        y=np.argmax(y,axis=1)#axis=1 means row direction
        t=np.argmax(t,axis=1)
        accuracy=np.sum(y==t)/float(x.shape[0]) # x.shape[0] is the line number
        return accuracy
    def numerical_gradient(self,x,t):
        loss_W=lambda W: self.loss(x,t)
        grads={}# dont forget use an empty dictionary
        grads['W1']=numerical_gradient_W(loss_W,self.params['W1'])
        grads['b1']=numerical_gradient(loss_W,self.params['b1'])# We should use different numerical_gradient function!!
        grads['W2']=numerical_gradient_W(loss_W,self.params['W2'])
        grads['b2']=numerical_gradient(loss_W,self.params['b2'])
        return grads

In [167]:
net = TwoLayerNet(input_size=784,hidden_size=100,output_size=10)
print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


In [107]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [169]:
x=np.random.rand(100,784)# forged input data
y=net.predict(x)
t=np.random.rand(100,10)#forged labels
grads=net.numerical_gradient(x,t)
print(grads['W1'].shape)
print(grads['b1'].shape)
print(grads['W2'].shape)
print(grads['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


In [113]:
print(y.shape) # because I forgot return in predict method :)

(100, 10)


In [123]:
print(f"params['b1'].shape is{net.params['b1'].shape} and params['b2'].shape is {net.params['b2'].shape}") 

params['b1'].shape is(100,) and params['b2'].shape is (10,)


In [131]:
# mnist project
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
mnist=fetch_openml('mnist_784',version=1)
X,y=mnist['data'],mnist['target'] # X is DataFrame, y is Series 
#the target y or t is of dtype='category'
#X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [145]:
# data preprocessing
npar_X,npar_y = X.values, y.astype(int).values
X_train,X_test,y_train,y_test = train_test_split(npar_X,npar_y,test_size=0.2,random_state=42)

In [147]:
encoder=OneHotEncoder(sparse_output=False) # get y one_hot encoded
y_train_oh=encoder.fit_transform(y_train.reshape(-1,1))
y_test_oh=encoder.fit_transform(y_test.reshape(-1,1))
print(y_train_oh.shape)
print(y_test_oh.shape)

(56000, 10)
(14000, 10)


In [149]:
print(X_train.shape)

(56000, 784)


In [151]:
#setting hyper parameter
iters_num=10000
train_size=X_train.shape[0]
batch_size=100
learning_rate=0.1
train_loss_list=[]#an empty list
network=TwoLayerNet(input_size=784,hidden_size=50,output_size=10)
#iteration
for i in range(iters_num):
    #get mini_batch
    batch_mask=np.random.choice(train_size,batch_size)
    X_batch=X_train[batch_mask]
    t_batch=y_train[batch_mask]
    #gradient calculation
    grad=network.numerical_gradient(X_batch,t_batch)
    #renew the weight and bias parameters
    for key in ('W1','b1','W2','b2'):
        network.params[key]-=learning_rate*grad[key]
    loss=network.loss(X_batch,t_batch)
    train_loss_list.append(loss)

ValueError: operands could not be broadcast together with shapes (100,) (100,10) 